# 案例演示

## 数据导入

In [69]:
import numpy as np
import pandas as pd
import graphviz
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import cross_val_score

train_data = pd.read_csv(r'train.csv')  #加载数据
test_data = pd.read_csv(r'test.csv')

训练集数据特征

In [70]:
train_data.info()   # 训练集数据特征

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [71]:
train_data.describe()   # 训练集统计情况

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


测试集数据特征

In [72]:
test_data.info()    # 测试集数据特征

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [73]:
test_data.describe()    # 测试集统计情况

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


## 数据清洗

### 训练集清洗

首先对训练集数据进行清洗，观察数据可知某些特征下对应样本数据局缺失。

1. Age中的空值可用平均年龄来填充。

In [74]:
train_data['Age'].fillna(train_data['Age'].mean(),inplace = True)

2. Cabin有大量的缺失值，在训练集和测试集中缺失率都比较高，无法补齐。
3. Embarked为登陆港口，可以根据港口属性补齐。可以看到港口为`S`类型的占比最高，可以考虑把缺失的港口用`S`港口填充。

In [75]:
train_data['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

可以看到港口为“S”类型的占比最高，可以考虑把缺失的港口用“S”港口填充。

In [76]:
train_data['Embarked'].fillna('S', inplace = True)

### 测试集清洗

1. Age中的空值可用平均年龄来填充。

In [77]:
test_data['Age'].fillna(test_data['Age'].mean(), inplace = True)

2. Fare中的空值可用平均票价来填充。

In [78]:
test_data['Fare'].fillna(test_data['Fare'].mean(),inplace = True)

3. Cabin有大量的缺失值，在训练集和测试集中缺失率都比较高，无法补齐

## 特征选择：选择分类结果有关键的特征

通过数据探索，发现PassengerId和Name对分类没有太大作用；Cabin有大量的缺失值，可以放弃；Ticket字段编码较乱，没有太大作用。其余字段可能和预测乘客的生存情况有关，通过分类器来处理。将剩余字段加入到特征向量中。

In [79]:
features = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
train_features = train_data[features]
train_labels = train_data['Survived']
test_features = test_data[features]

特征值有一些是字符串，不方便后续的运算，需要把它们转化为数值类型。（数据优化）

1. Sex 有male和female两种类型，可以把它变成 Sex=male 和 Sex = female， 数值用0或1来表示。
2. Embarked 有S 、C 、Q三种类型，可以把它变成Embarked= S 、 Embarked= C、Embarked= Q，数值用0或1来表示。

可以使用sklearn 特征选择中的 DictVectorizer类，用它可以处理符号化的对象，将符号转化为数字0或1进行表示。其次，fit_transform可以将特征向量转化为特征矩阵，通过dv.feature_names_ 属性值查看转化后的属性。

In [80]:
dv = DictVectorizer(sparse = False)
train_features = dv.fit_transform(train_features.to_dict(orient= 'record'))
dv.feature_names_

/var/folders/85/kk7q3wb94jz88s0bdn7qntkw0000gn/T/ipykernel_6359/2932227385.py:2: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  train_features = dv.fit_transform(train_features.to_dict(orient= 'record'))


['Age',
 'Embarked=C',
 'Embarked=Q',
 'Embarked=S',
 'Fare',
 'Parch',
 'Pclass',
 'Sex=female',
 'Sex=male',
 'SibSp']

## 决策树模型：使用ID3算法构造决策树

创建决策树时，设置`criterion = 'entropy'`，然后使用`fit`进行训练，将特征值矩阵和分类结果作为参数传入，得到决策树分类器。

In [81]:
clf = DecisionTreeClassifier(criterion='entropy')
clf.fit(train_features, train_labels)   # 特征加入

DecisionTreeClassifier(criterion='entropy')

## 模型评估和预测

在预测中，首选需要得到测试集的特征值矩阵，然后使用训练好的决策树进行预测。

In [82]:
test_features = dv.transform(test_features.to_dict(orient= 'record'))
pred_labels = clf.predict(test_features)

/var/folders/85/kk7q3wb94jz88s0bdn7qntkw0000gn/T/ipykernel_6359/2116926651.py:1: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  test_features = dv.transform(test_features.to_dict(orient= 'record'))


在模型评估中，决策树提供了score函数可以直接得到准确率。但由于我们的测试集中并没有真实的生存状况的结果，只能使用训练集中的数据进行模型评估

In [83]:
acc_decision_tree = round(clf.score(train_features, train_labels), 6)
acc_decision_tree

0.982043

预测结果保存到文件中。

In [84]:
test_data['Survived'] = pred_labels
test_data.to_csv(r'test1.csv')  #预测后的测试集导出

使用Graphviz可视化工具导出决策树

In [85]:
dot_data = tree.export_graphviz(clf, out_file=None)
graph = graphviz.Source(dot_data)
graph.view()

'Source.gv.pdf'

## K折交叉验证原理

1. 将数据集平均分割成K个等份。
2. 使用1份的数据作为测试数据，其余作为训练数据。
3. 计算测试准确率。
4. 使用不同的测试集，重复2、3步骤。
5. 训练k次，最后将k次的测试准确率求平均值，作为对未知数据预测准确率的估计。

在sklearn的model_selection 模型选择中提供了cross_val_score函数

In [86]:
np.mean(cross_val_score(clf, train_features, train_labels, cv=10))

0.7845817727840201